## Goroutines

- A goroutine is a lightweight thread managed by the Go runtime.
    ```go
    go f(x, y, z)
    ```

- e.g.
    ```go
    package main

    func main() {
        go func() {
            println("abc")
        }()

        func() {
            println("xyz")
        }()
    }
    ```

- output

    ```go
    xyz
    ```
    
- This has output only `xyz` because the `main` function executes the second function and exits even before the 1st function(go routine) starts/executes.

- e.g.

    ```go
    func main() {
        go func() {
            println("abc")
        }()

        go func() {
            println("xyz")
        }()
	
        time.Sleep(time.Second * 1)	
    }
    ```
    
- If the above program has no `Sleep` function call then there will be no output shown just because `main` will exit before others, introducing `time` helps.

- If `GOMAXPROCS` value is one and we have multiple go routines then all these go routines will be spawned as threads running concurrently, no parallelism, but if the `GOMAXPROS` > 1 then this means newer go routine threads will be allocated to other processor and will be run in parallel.

- e.g.

    ```go
    func main() {
    	//runtime.GOMAXPROCS(2)                          // Uncomment for 3rd output                   
        go func() {
            for i := 0; i < 5; i++ {
                println("abc")
                //time.Sleep(time.Millisecond * 10)      // Uncomment for 2nd and 3rd output
            }
        }()

        go func() {
            for i := 0; i < 5; i++ {
                println("xyz")
                //time.Sleep(time.Millisecond * 10)      // Uncomment for 2nd and 3rd output     
            }
        }()
        time.Sleep(time.Second * 1)
    }
    ```

-    output:
    ```bash
    abc                   ## This output is like this because each function is running
    abc                   ## concurrently so they execute comletely in very short amount
    abc                   ## of time, even if they are two threads
    abc
    abc
    xyz
    xyz
    xyz
    xyz
    xyz
    ```

-    output with `time.Sleep`
    ```bash
    abc               ## here we can see a proper multi-threaded output because each thread/routine
    xyz               ## sleeps for some time so the runtime runs other thread while one sleeps
    xyz
    abc
    abc
    xyz
    xyz
    abc
    abc
    xyz
    
    ```

- output with `time.Sleep` and `runtime.GOMAXPROCS`

    ```bash
    abc
    xyz           ## Here actual parallel processing has been done, so here output can be random
    abc           ## sometimes two things can be printed on the same line.
    xyz
    abc
    xyz
    xyz
    abc
    xyz
    abc
    ```

- Things that are done sequentially can be spawned in different routines.
- Scenario: Downloading some data from some website and then parsing it and showing the results, can be paralleled. Becasue most of the times, the network io is the blocking call, so the execution speeds can be increased by making it parallel.

### GOMAXPROCS

Why doesn't my multi-goroutine program use multiple CPUs?

The number of CPUs available simultaneously to executing goroutines is controlled by the `GOMAXPROCS` shell environment variable. In earlier releases of Go, the default value was 1, but as of Go 1.5 the default value is the number of cores available. Therefore programs compiled after 1.5 should demonstrate parallel execution of multiple goroutines. To change the behavior, set the environment variable or use the similarly-named function of the `runtime` package to configure the run-time support to utilize a different number of threads.

Programs that perform parallel computation might benefit from a further increase in `GOMAXPROCS`. However, be aware that concurrency is not parallelism.

Ref: 

   - Go FAQ: https://golang.org/doc/faq#Why_no_multi_CPU
   - Stack: http://stackoverflow.com/a/17868720